In [45]:
from faker import Faker
import numpy as np 
import pandas as pd
import random
import csv
import lorem

In [46]:
url = 'c:/Users/lauis/Downloads/SDM-lab03/generationData/newCSV/basicCSV/'
# Read all data from articles 
## Article headers
with open(url+'output_article_header.csv') as f:
    article_header = f.readline().split(';')

## Articles in dataset
articles = pd.read_csv(url+'output_article.csv', header=0, names=article_header, sep=';');

C:\Users\lauis\AppData\Local\Temp\ipykernel_28600\3117579325.py:8: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,14,18,19,20,21,22,23,24,25,26,30,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv(url+'output_article.csv', header=0, names=article_header, sep=';');


In [47]:
# Selecting only 5000 articles
art_sel = articles[:5000]

In [48]:
art_sel['sup:string[]'] = np.random.choice(['J', 'C'], size=len(art_sel), p=[0.5, 0.5])

# Identify categorical variables, 'publisher:string'
categorical_vars = ['author:string[]', 'article:ID','author-orcid:string[]']
# Remove rows with missing values in categorical variables
art_sel = art_sel.dropna(subset=categorical_vars)

C:\Users\lauis\AppData\Local\Temp\ipykernel_28600\2685938444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_sel['sup:string[]'] = np.random.choice(['J', 'C'], size=len(art_sel), p=[0.5, 0.5])


In [49]:
art_sel.rename(columns={'article:ID': "DOI",'author:string[]':"AuthorName",'author-aux:string': "author-aux",
       'author-orcid:string[]': "AuthorID", 'booktitle:string': "booktitle", 'cdate:date': "cdate",
       'cdrom:string': "cdrom", 'cite:string[]': "cite", 'cite-label:string[]': "cite-label",
       'crossref:string': "crossref", 'editor:string[]': "editor", 'editor-orcid:string[]': "editor-orcid",
       'ee:string[]': "ee", 'ee-type:string[]':"ee-type", 'i:string[]':"i", 'journal:string': "journal",
       'key:string': "key", 'mdate:date': "mdate", 'month:string':"month", 'note:string[]':"note",
       'note-label:string':"note-label", 'note-type:string[]':"note-type", 'number:string': "number",
       'pages:string': "pages", 'publisher:string':"publisher", 'publnr:string':"publnr", 'publtype:string':"publtype",
       'sub:string[]':"sub", 'sup:string[]':"VenueType", 'title:string':"Title", 'title-bibtex:string':"title-bibtex",
       'tt:string[]':"tt", 'url:string[]':"url", 'volume:string':"VolumeNr", 'year:int': "YearPublished"
      }, inplace=True)

In [50]:

ref_C = ["FullPaper", "ShortPaper", "DemoPaper", "Poster"]
ref_O = ["FullPaper", "ShortPaper", "DemoPaper"]

def assign_citation_type(x):
    if x['VenueType'] != 'C':
        return random.choice(ref_O)
    else:
        return random.choice(ref_C)

art_sel['PaperType'] = art_sel.apply(assign_citation_type, axis=1)

In [51]:
## Generating abstracts using Lorem library
abstracts = []

for i in range(art_sel.shape[0]):
    abstracts.append(lorem.paragraph())
        
art_sel['Abstract'] = abstracts

In [52]:
fake = Faker()

def assign_editor_type(x):
    if x['VenueType'] != 'C':
        return fake.name()
    else:
        return None

art_sel['EditorName'] = art_sel.apply(assign_editor_type, axis=1)
art_sel['chair']=None

def assign_chair_type(x):
    if x['VenueType'] == 'C':
        return fake.name()
    else:
        return None

art_sel['ChairName'] = art_sel.apply(assign_chair_type, axis=1)

In [53]:
def assign_conference_name(x):
    if x['VenueType'] == 'C':
        return fake.name()
    else:
        return None
ref_O = ["W", "S", "E", "R"]
def assign_conference_type(x):
    if x['VenueType'] != 'C':
        return random.choice(ref_O)
    else:
        return None
art_sel['ConfName'] = art_sel.apply(assign_conference_name, axis=1)
art_sel['ConfType'] = art_sel.apply(assign_conference_type, axis=1)

In [54]:
def assign_journal_name(x):
    if x['VenueType'] == 'J':
        return fake.name()
    else:
        return None
art_sel['JournalName'] = art_sel.apply(assign_journal_name, axis=1)

In [55]:
### Reviews
## articles for reviews 
art_rev_selected = art_sel[['DOI', 'AuthorName', 'Title']]

# Creating list of reviewers
split_authors = art_rev_selected['AuthorName'].str.split("|", expand=True)
authors_list = []

for i in split_authors[split_authors.columns[0]].values.tolist():
    authors_list.append(i)
        
my_auth_list = list(set(list(authors_list)))
# remove nan from first column
my_auth_list = my_auth_list[1:]
## Duplicate art_rev_selected thrice because we assume that every articles must have at least 3 reviews
art_rev_selected = pd.concat([art_rev_selected]*2, ignore_index=True)

In [56]:
## Assigning authors list to reviews
random.seed(42)

# define list of values to randomly assign
my_list = my_auth_list
# define function to randomly assign value from list to new column, avoiding strings
def assign_value(row, my_list):
    new_value = random.choice(my_list)
    while any([str(new_value) in str(x) for x in row.values]):
        new_value = random.choice(my_list)
    return new_value

# apply function to create new column with randomly assigned values
art_sel['ReviewerName1'] = art_sel.apply(lambda row: assign_value(row, my_list), axis=1)
art_sel['ReviewerName2'] = art_sel.apply(lambda row: assign_value(row, my_list), axis=1)

# Create reviews using Lorem library
def assign_review(row):
    return lorem.sentence()
        
art_sel['ReviewText1'] = art_sel.apply(assign_review, axis=1)
art_sel['ReviewText2'] = art_sel.apply(assign_review, axis=1)

art_sel['ReviewDecision1'] = np.random.choice(['A','R'], size=len(art_sel), p=[0.80, 0.20])
art_sel['ReviewDecision2'] = np.random.choice(['A','R'], size=len(art_sel), p=[0.80, 0.20])


In [57]:
def assign_FinalDecision(x):
    if x['ReviewDecision1'] == 'A' and x['ReviewDecision2'] == 'A' :
        return 'A'
    else:
        return 'R'
art_sel['FinalDecision'] = art_sel.apply(assign_FinalDecision, axis=1)

In [58]:
art_sel['YearPublished'] = art_sel['YearPublished'].astype(int)

In [59]:
# Adding more keywords from exercises so that least 90% is contained from this 
db_keywords = ['data management', 'indexing', 'data modeling', 'big data', 'data processing',
            'data storage', 'data querying']
# Generate KEYWORDS from titles/abstract of articles to add for the community
def get_keywords(x):
    split_title = str(x["Title"]).split(" ")
    kwords = []
    ## words with lenght 15
    for i in split_title:
        if len(str(i))==15:
            kwords.append(i)
    
    # Combine list 
    combined_list = db_keywords + my_list
    return '|'.join(combined_list)
    
art_sel['Keywords'] = art_sel.apply(get_keywords, axis=1)

In [60]:
# create new column with randomly assigned values area
def assign_area(x):
    return random.choice(db_keywords)

art_sel['Area'] = art_sel.apply(assign_area, axis=1)

In [61]:
indexnr=[]
volumenr=[]
indexnrp=range(100)
volumenrp=range(500)
for x in art_sel['VenueType']:
    temp_ind=None
    temp_volume=None
    if x == 'J':
        temp_volume=str(int(random.choice(volumenrp)))
    else:
        temp_ind=str(int(random.choice(indexnrp)))
    indexnr.append(temp_ind)
    volumenr.append(temp_volume)

art_sel['IndexNr'] =indexnr
art_sel['VolumeNr'] = volumenr


In [62]:
!pip install -U scikit-learn scipy matplotlib


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
from sklearn.model_selection import train_test_split

# Split for cite and citation
# split the dataframe into two parts i.e 0.33% for cite
art_citation, art_cite = train_test_split(art_sel, test_size=0.33, random_state=42)
# Ensure articles in citation are written before cite articles
values = [2017, 2018, 2019]

## randomly assign the values as a new column in the DataFrame
art_citation['YearPublished'] = np.random.RandomState(42).choice(values, len(art_citation))

# create a list of values to assign randomly from 2021-2022
values = [2020, 2021, 2022]
art_cite['YearPublished'] = np.random.RandomState(42).choice(values, len(art_cite))

## Selecting citation
cite_cita = art_citation[['author-aux', 'DOI']]

## Replicate the citation df so that each article has at least 2 citations
cite_cita = pd.concat([cite_cita]*1, ignore_index=True)
## randomly assign cite articles to citation
ref_in_art_cite = list(set(list(art_cite['DOI'])))
# Define function to randomly assignIDs
def assign_no():
    return random.choice(ref_in_art_cite)

art_sel['Citation_DOI'] = art_sel.apply(lambda x: assign_no(), axis=1)

In [64]:
art_sel.columns

Index(['DOI', 'AuthorName', 'author-aux', 'AuthorID', 'booktitle', 'cdate',
       'cdrom', 'cite', 'cite-label', 'crossref', 'editor', 'editor-orcid',
       'ee', 'ee-type', 'i', 'journal', 'key', 'mdate', 'month', 'note',
       'note-label', 'note-type', 'number', 'pages', 'publisher', 'publnr',
       'publtype', 'sub', 'VenueType', 'Title', 'title-bibtex', 'tt', 'url',
       'VolumeNr', 'YearPublished', 'PaperType', 'Abstract', 'EditorName',
       'chair', 'ChairName', 'ConfName', 'ConfType', 'JournalName',
       'ReviewerName1', 'ReviewerName2', 'ReviewText1', 'ReviewText2',
       'ReviewDecision1', 'ReviewDecision2', 'FinalDecision', 'Keywords',
       'Area', 'IndexNr', 'Citation_DOI'],
      dtype='object')

In [65]:
finalcolumns=['DOI', 'AuthorName', 'AuthorID', 'VenueType', 'Title',
 'YearPublished', 'PaperType', 'Abstract', 'EditorName',
        'ChairName', 'ConfName', 'ConfType', 'JournalName',
       'ReviewerName1', 'ReviewerName2', 'ReviewText1', 'ReviewText2',
       'ReviewDecision1', 'ReviewDecision2', 'FinalDecision', 'Keywords', 'VolumeNr',
       'IndexNr','Citation_DOI','Area']

art_sel_final=art_sel[finalcolumns]

# Save to CSV
## Articles instances 
art_sel_final.to_csv(path_or_buf='datafinal/data_general.csv', index=False, header=True)

In [66]:
art_sel_final.head()

,DOI,AuthorName,AuthorID,VenueType,Title,YearPublished,PaperType,Abstract,EditorName,ChairName,...,ReviewText1,ReviewText2,ReviewDecision1,ReviewDecision2,FinalDecision,Keywords,VolumeNr,IndexNr,Citation_DOI,Area
5,4103150,Hongjian Fan|Kotagiri Ramamohanarao,0000-0003-3304-9268,C,Patterns Based Classifiers.,2007,DemoPaper,Labore quiquia consectetur non. Sit ut quaerat...,None,Sheila Henderson,...,Dolore est est neque dolor.,Modi non est ipsum.,A,R,R,data management|indexing|data modeling|big dat...,None,67,6700625,data querying
8,4104163,Morgan Ericsson,0000-0003-1173-5187,C,The Effects of XML Compression on SOAP Perform...,2007,Poster,Labore voluptatem dolor tempora ipsum quaerat....,None,Jonathan Benitez,...,Quiquia quiquia quiquia eius.,Etincidunt non tempora magnam aliquam quaerat ...,A,A,A,data management|indexing|data modeling|big dat...,None,0,6698457,big data
9,4104216,David L. Martin 0001|Deborah L. McGuinness|Dre...,0000-0001-7037-4567,C,Bringing Semantics to Web Services with OWL-S.,2007,FullPaper,Aliquam dolorem dolor non. Sed dolore numquam ...,None,Angel Harrell,...,Non modi porro non aliquam amet.,Consectetur voluptatem dolor porro eius velit.,A,A,A,data management|indexing|data modeling|big dat...,None,41,6700352,big data
12,4106084,Andrzej Cichocki|Dimitrios Georgakopoulos|Dona...,0000-0001-7880-2140,J,Event-driven Video Awareness Providing Physica...,2007,ShortPaper,Voluptatem consectetur dolorem labore magnam e...,Joseph Little,None,...,Magnam dolor modi dolorem quisquam velit.,Eius neque dolor consectetur sit.,A,A,A,data management|indexing|data modeling|big dat...,126,None,6698473,data querying
15,4108256,Jeffrey Scott Vitter|Lipyeow Lim|Min Wang 0001...,0000-0001-7970-6118,C,Efficient Update of Indexes for Dynamically Ch...,2007,DemoPaper,Tempora non est numquam neque etincidunt non. ...,None,Robert Johnson,...,Eius quisquam velit porro velit adipisci magna...,Numquam labore modi tempora magnam modi quaera...,R,A,R,data management|indexing|data modeling|big dat...,None,56,6698538,data storage
